In [3]:
from bs4 import BeautifulSoup
import csv

# Path to your local HTML file
file_path = '/home/student/Desktop/python/coursera/Courses.html'

# Open and read the file
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Select all course cards - adjust this selector if needed based on your HTML
course_cards = soup.select('li')

courses_data = []

for card in course_cards:
    # Course Name
    course_name_tag = card.select_one('a.cds-CommonCard-titleLink h3')
    course_name = course_name_tag.text.strip() if course_name_tag else None

    # Provider
    provider_tag = card.select_one('p.cds-ProductCard-partnerNames')
    provider = provider_tag.text.strip() if provider_tag else None

    # Skills you'll gain
    skills_tag = card.find('p', string=lambda t: t and "Skills you'll gain" in t)
    if not skills_tag:
        for p in card.select('p'):
            strong_tag = p.find('strong')
            if strong_tag and "Skills you'll gain" in strong_tag.text:
                skills_tag = p
                break
    skills = skills_tag.text.replace("Skills you'll gain:", "").strip() if skills_tag else None

    # Rating
    rating_div = card.find('div', attrs={'aria-label': 'Rating'})
    rating = None
    if rating_div:
        rating_span = rating_div.find('span')
        rating = rating_span.text.strip() if rating_span else None

    # Total Reviews
    total_reviews = None
    review_divs = card.select('div.css-vac8rf')
    for div in review_divs:
        text = div.text.strip()
        if "reviews" in text:
            total_reviews = text
            break

    # Metadata (Level and Duration)
    metadata_p = card.select_one('div.cds-CommonCard-metadata p.css-vac8rf')
    level = None
    duration = None
    if metadata_p:
        parts = [part.strip() for part in metadata_p.text.split('·')]
        if len(parts) >= 1:
            level = parts[0]
        if len(parts) >= 3:
            duration = parts[2]

    if course_name:  # Only add if course name exists
        courses_data.append({
            'Course Name': course_name,
            'Provider': provider,
            'Skills': skills,
            'Rating': rating,
            'Total Reviews': total_reviews,
            'Level': level,
            'Duration': duration
        })

# Save to CSV
csv_file = '/home/student/Desktop/python/coursera/courses_html.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Course Name', 'Provider', 'Skills', 'Rating', 'Total Reviews', 'Level', 'Duration'])
    writer.writeheader()
    writer.writerows(courses_data)

print(f"Data saved to {csv_file}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/student/Desktop/python/coursera/Courses.html'